In [12]:
import torch
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import tqdm
import nltk
import os
import sys
UNK = '<unk>'
PAD = '<pad>'
NULL = 'NULL'
START_TAG = "<START>"
STOP_TAG = "<STOP>"

In [13]:
class Loader:
    # word, tag, character, orth, label
    def __init__(self):
        self.train_label_path = "./data/train/train.txt"
        self.dev_label_path = "./data/dev/dev.txt"
        self.test_path = "./data/test/test.nolabels.txt"
        self.word_to_id = None
        self.id_to_word = None
        self.tag_to_id = None
        self.id_to_tag = None
        self.label_to_id = None
        self.id_to_label = None
        self.char_to_id = None
        self.id_to_char = None
        self.orth_to_id = None
        self.id_to_orth = None
        self.max_len = 52

    def load_data(self,mode):
        sentences, tags, labels = self.build_sentences(mode)
        if mode == 'train':self.build_vocab(sentences,tags,labels)
        word_input,tag_input,char_input,orth_input,label_output = self.build_data(sentences, tags, labels, mode)
        sentence_len = []
        for sentence in sentences:sentence_len.append(len(sentence))
        sentence_len = np.array(sentence_len)
        return sentence_len,word_input,tag_input,char_input,orth_input,label_output

    def build_data(self,sentences, tags, labels, mode):
        word_input = np.zeros((len(sentences),self.max_len)).astype(int)
        tag_input = np.zeros(word_input.shape).astype(int)
        label_output = np.zeros(word_input.shape).astype(int)
        char_input = np.zeros((len(sentences), self.max_len, len(self.char_to_id))).astype(int)
        orth_input = np.zeros((len(sentences),self.max_len)).astype(int)
        for i,sentence in enumerate(sentences):
            for j,word in enumerate(sentence):
                if word in self.word_to_id:word_input[i,j] = self.word_to_id[word]
                else:word_input[i,j] = self.word_to_id[UNK]
                orthword = self.orthographic(word)
                if orthword in self.orth_to_id: orth_input[i,j] = self.orth_to_id[orthword]
                else: orth_input[i,j] = self.orth_to_id[UNK]
                for k,char in enumerate(word):
                    if char in self.char_to_id: char_input[i,j,self.char_to_id[char]]+=1
                    else:char_input[i,j,self.char_to_id[UNK]]+=1
        for i,sentence_tag in enumerate(tags):
            for j,tag in enumerate(sentence_tag):
                if tag in self.tag_to_id:tag_input[i,j] = self.tag_to_id[tag]
        if mode=='test':return word_input, tag_input, char_input, orth_input, None
        for i,sentence_label in enumerate(labels):
            for j,label in enumerate(sentence_label):
                label_output[i,j] = self.label_to_id[label]
        return word_input, tag_input, char_input, orth_input, label_output

    def build_vocab(self,sentences,tags,labels):
        # build word-id char-id orth-id dictionary
        counts = {}
        char_to_id = {}
        orth_to_id = {}
        charidx = 0
        orthidx = 0
        for sentence in sentences:
            for word in sentence:
                if word not in counts:counts[word]=1
                else: counts[word]+=1
                # orthographic
                orthword = self.orthographic(word)
                if orthword not in orth_to_id:
                    orth_to_id[orthword] = orthidx
                    orthidx += 1
                # character
                for char in word:
                    if char not in char_to_id:
                        char_to_id[char] = charidx
                        charidx+=1
        char_to_id[UNK] = charidx
        id_to_char = {v:k for k,v in char_to_id.items()}
        self.char_to_id = char_to_id
        self.id_to_char = id_to_char
        orth_to_id[UNK] = orthidx
        id_to_orth = {v:k for k,v in orth_to_id.items()}
        self.orth_to_id = orth_to_id
        self.id_to_orth = id_to_orth
        sorted_counts = sorted(counts.items(),key=lambda x:(-x[1],x[0]))
        word_to_id = {}
        idx = 0
        for word,count in sorted_counts:
            if word not in word_to_id:
                word_to_id[word] = idx
                idx += 1
        word_to_id[UNK] = idx
        id_to_word = {v:k for k,v in word_to_id.items()}
        self.word_to_id = word_to_id
        self.id_to_word = id_to_word

        # build label-id dictionary
        label_to_id={START_TAG:0}
        idx=1
        for sentence_label in labels:
            for label in sentence_label:
                if label not in label_to_id:
                    label_to_id[label] = idx
                    idx+=1
        label_to_id[STOP_TAG]=idx
        id_to_label = {v:k for k,v in label_to_id.items()}
        self.label_to_id = label_to_id
        self.id_to_label = id_to_label

        # build tag-id dictionary
        tag_to_id = {NULL:0}
        idx=1
        for sentence_tag in tags:
            for tag in sentence_tag:
                if tag not in tag_to_id:
                    tag_to_id[tag] = idx
                    idx += 1
        id_to_tag = {v:k for k,v in tag_to_id.items()}
        self.tag_to_id = tag_to_id
        self.id_to_tag = id_to_tag

    def orthographic(self, word):
        ortho_word = ''
        for char in word:
            if char.isupper():
                ortho_word += 'C'
            elif char.islower():
                ortho_word += 'c'
            elif char.isdigit():
                ortho_word += 'n'
            else:
                ortho_word += 'p'
        return ortho_word

    def build_sentences(self,mode):
        sentences = []
        labels = []
        tags = []
        if mode=="test":
            with open(self.test_path,encoding='utf-8') as f:
                sentence=[]
                for line in f:
                    line = line.strip()
                    if not line:
                        sentences.append(sentence)
                        tags.append([pair[1] for pair in nltk.pos_tag(sentence)])
                        sentence = []
                        continue
                    sentence.append(line.lower())
        else:
            if mode=="train": file = open(self.train_label_path,encoding='utf-8')
            else: file = open(self.dev_label_path,encoding='utf-8')
            sentence=[]
            label=[]
            for line in file:
                line = line.split()
                if not line:
                    sentences.append(sentence)
                    tags.append([pair[1] for pair in nltk.pos_tag(sentence)])
                    labels.append(label)
                    sentence=[]
                    label=[]
                    continue
                word,wordlabel = line
                sentence.append(word.lower())
                label.append(wordlabel)
            file.close()
        return sentences,tags,labels

In [14]:
def argmax(vec):
    _, idx = torch.max(vec, 1)
    return idx.item()

def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # calculate in log domain
        # feats is len(sentence) * tagset_size
        # initialize alpha with a Tensor with values all equal to -10000.

        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                    self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)  # all the words' all tags score
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [15]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile, 'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.", len(model), " words loaded!")
    return model

def build_worddict_embedding(gloveFile,worddict):
    print("Build loader's worddict embedding from glove")
    glove = loadGloveModel(gloveFile)
    embedding = []
    count = 0
    dim = np.sqrt(3.0 /glove['hello'].shape[0])
    for k,v in worddict.items():
        if v in glove:
            embedding.append(glove[v])
        else:
            count+=1
            y = np.random.uniform(-dim,dim,glove['hello'].shape[0])
            embedding.append(y)
    embedding = np.array(embedding)
    print(embedding.shape)
    print("There are ",count," random initial embedding")
    return embedding

def calculatef1(y,p,id_to_class):
    true_pos = 0
    false_pos = 0
    false_neg = 0
    for y_seq,p_seq in zip(y,p):
        for y_id,p_id in zip(y_seq,p_seq):
            y_ = id_to_class[y_id]
            p_ = id_to_class[p_id]
            if y_ != 'O':
                if p_ == y_:
                    true_pos += 1
                elif p_ == 'O':
                    false_neg += 1
            elif p_ != y_:
                false_pos += 1
    prec = true_pos / (true_pos + false_pos) if true_pos + false_pos != 0 else 0
    recall = true_pos / (true_pos + false_neg) if true_pos + false_neg != 0 else 0
    f1 = 2 * prec * recall / (prec + recall) if prec + recall != 0 else 0
    return f1, prec, recall

In [ ]:
print("Load Data from file")
loader = Loader()
train_len, train_word, train_tag, train_char, train_orth, train_label = loader.load_data('train')
print("Train: ",train_word.shape[0])
dev_len,dev_word, dev_tag, dev_char, dev_orth, dev_label = loader.load_data('dev')
print("Dev: ",dev_word.shape[0])
test_len, test_word, test_tag, test_char, test_orth, test_label = loader.load_data('test')
print("Test: ", test_word.shape[0])

EMBEDDING_DIM = 200
HIDDEN_DIM = 100
LR = 0.001

model = BiLSTM_CRF(len(loader.word_to_id), loader.label_to_id, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=LR, weight_decay=1e-4)

for epoch in range(30):
    print("Epoch",epoch)
    i=0
    for sentence, labels in tqdm.tqdm(zip(train_word,train_label)):
        model.zero_grad()
        sentence_in = torch.tensor(sentence[:train_len[i]], dtype=torch.long)
        targets = torch.tensor(labels[:train_len[i]], dtype=torch.long)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()
        i+=1
        if(i%1000==0):print(loss)
    print()


Load Data from file
Train:  2394
Dev:  959


2it [00:00, 16.38it/s]

Test:  2377
Epoch 0


1003it [00:51, 19.44it/s]

tensor([ 0.8427])


...

2003it [01:12, 27.63it/s]

tensor([ 0.2418])


2394it [01:27, 27.51it/s]
3it [00:00, 29.56it/s]


Epoch 7


101it [00:03, 27.49it/s]

In [17]:
dev_predict=[]
dev_y=[]
i=0
for sentence, labels in zip(dev_word, dev_label):
    dev_y.append(labels[:dev_len[i]])
    dev_predict.append(model(torch.tensor(sentence[:dev_len[i]], dtype=torch.long))[1])
    i+=1
f1, prec, recall = calculatef1(dev_y,dev_predict,loader.id_to_label)
print(f1, prec, recall)

0.17177914110429449 0.7692307692307693 0.09668508287292818


In [ ]:
test_predict = []
i=0
for sentence in test_word:
    test_predict.append(model(torch.tensor(sentence[:test_len[i]], dtype=torch.long))[1])
    i+=1
file = open("test1.out",'w',encoding='utf-8')
for i in test_predict:
    for j in i:
        file.write(loader.id_to_label[j])
        file.write('\n')
    file.write('\n')
file.close()